In [ ]:
#default_exp update

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

# Update
update the database

In [ ]:
#export
from s3bz.s3bz import S3
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict, hashDict, saveStringToFile, loadStringFromFile, saveDictToFile, loadDictFromFile
from nicHelper.exception import errorString
from dict_hash import dict_hash, sha256
from base64 import b64encode, b64decode
from dataclasses_json import dataclass_json, Undefined, CatchAll
from dataclasses import dataclass
from typing import List
from datetime import datetime
import pandas as pd
import os, logging

In [ ]:
#hide
from villaProductDatabase.database import ProductDatabase
import logging
logging.basicConfig(level=logging.INFO)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
import os
DBHASHLOCATION = '/tmp/database.hash'
DBCACHELOCATION = '/tmp/database.cache'
DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
  print(DAX_ENDPOINT)
except KeyError as e:
  print(f'dax endpoint missing {e}')

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
class Updater:
  pass

In [ ]:
class Tester( Updater, ProductDatabase):
  class Meta:
    table_name = os.environ['DATABASE_TABLE_NAME']
    region = os.environ['REGION']
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass

## Update with dict

In [ ]:
#export
@add_class_method(Updater)
def updateWithDict(cls, originalObject:Updater, inputDict:dict ):
  data = originalObject.data
  data.update(inputDict)
  return cls.fromDict(data)

In [ ]:
# test
item = next(Tester.query('0000009'))
print(f'engname is {item.data.get("pr_engname")}')

updatedItem = Tester.updateWithDict(item,{'pr_engname':'testName'})
assert updatedItem.data.get('pr_engname') == 'testName'
print(updatedItem.data.get('pr_engname'))

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


engname is JOHNNIE WALKER RED 70 CL.
testName


## update with list of dict

In [ ]:
#export
@dataclass_json(undefined=Undefined.INCLUDE)
@dataclass
class Product:
  iprcode: str
  cprcode: str
  data: CatchAll
@dataclass_json
@dataclass
class ValueUpdate:
  items: List[Product]
#export
def chunks(l, n): return [l[x: x+n] for x in range(0, len(l), n)]
@add_class_method(Updater)
def valueUpdate2(cls, inputs:List[dict]):
    ''' 
      check for difference and batch update the changes in product data
    '''
    t0 = datetime.now()
    ### validate input
    try:
      validInputs = ValueUpdate.from_dict(inputs).to_dict().get('items')
    except Exception as e:
      raise KeyError(f'input failed validation {e}')
      return
    
    itemsUpdated = {'success':0, 'failure': 0, 'skipped': 0 ,'failureMessage':[], 'timetaken(ms)': 0}
    t0 = datetime.now()

    logging.info(f'there are {len(validInputs)} products to update')

    print(f'input validated {(datetime.now()-t0).total_seconds()*1000} ms')
    ##### dividing input into batch of 500
    inputBatches = chunks(validInputs, 500)
    print(f'divided into chunks {(datetime.now()-t0).total_seconds()*1000} ms')
    db = cls.loadFromCache().fillna('none')
    print(f'get all from s3 {(datetime.now()-t0).total_seconds()*1000} ms')
    
    for inputBatch in inputBatches:
      with cls.batch_write() as batch:
        # loop through each product
        for input_ in inputBatch:
          iprcode = input_['iprcode']
          cprcode = input_['cprcode']

          ##### check if product is in the database, if not, create an empty class with the product code
          if 'cprcode' in db.columns:
            incumbentSeries = db[db['cprcode']==cprcode].iloc[0]
            if incumbentSeries.any: 
              print(f'incumbentSeries is type {type(incumbentSeries)}')
              incumbentItem = cls.fromSeries(incumbentSeries)
            else: incumbentItem = cls.fromDict({'iprcode': iprcode, 'cprcode': cprcode})
          else: incumbentItem = cls.fromDict({'iprcode': iprcode, 'cprcode': cprcode})
            
          ##### make a copy of original data
          originalData = incumbentItem.data.copy()
          ###### update data
          updatedData = cls.updateWithDict(incumbentItem, input_)

          logging.info(f'incumbentBr is {incumbentItem.iprcode}\n, prcode is {iprcode}')

          # check for difference
          try:
            if updatedData.data != originalData:
              logging.info(f'product {iprcode} has changed from \n{originalData} \n{updatedData.data}')
              batch.save(updatedData)
              itemsUpdated['success'] += 1
            else:
              logging.info(f'no change for {iprcode}')
              itemsUpdated['skipped'] += 1
          except Exception as e:
            itemsUpdated['failure'] += 1
            itemsUpdated['failureMessage'].append(e)
          
        # log time taken
        itemsUpdated['timetaken(ms)'] = (datetime.now()- t0).total_seconds()*1000
    return itemsUpdated

In [ ]:
%%time
sampleProducts = [{ 'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] #ProductDatabase.valueUpdate({'items':sampleProducts})
products = ValueUpdate.from_dict({'items':sampleProducts}).to_dict()
Tester.valueUpdate2(products)
Tester.saveRemoteCache()

INFO:root:there are 1 products to update
INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint
INFO:root:incumbentBr is 0171670
, prcode is 0171670
INFO:root:product 0171670 has changed from 
{'iprcode': '0171670', 'cprcode': '0171670'} 
{'iprcode': '0171670', 'cprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}


input validated 2.3040000000000003 ms
divided into chunks 2.61 ms
loading hashkey allData-hash
error downloading file An error occurred (404) when calling the HeadObject operation: Not Found
local loading error'list' object has no attribute 'get', loading remote hash
locding remtoe failed An error occurred (404) when calling the HeadObject operation: Not Found returning blank df
get all from s3 140.471 ms
CPU times: user 56.9 ms, sys: 3.55 ms, total: 60.4 ms
Wall time: 153 ms


{'success': 1,
 'failure': 0,
 'skipped': 0,
 'failureMessage': [],
 'timetaken(ms)': 145.63899999999998}

In [ ]:
db = Tester.loadFromCache()
db

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


loading hashkey allData-hash
error downloading file An error occurred (404) when calling the HeadObject operation: Not Found
local loading error'list' object has no attribute 'get', loading remote hash
locding remtoe failed An error occurred (404) when calling the HeadObject operation: Not Found returning blank df


""


In [ ]:
sampleProducts = [{ 'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] #ProductDatabase.valueUpdate({'items':sampleProducts})
product = sampleProducts[0]
Tester.fromDict(product).toSeries()

cprcode                          0171670
iprcode                          0171670
oprcode                          0171670
ordertype                              Y
pr_abb              JIRAPAT YOUNG KALE 2
pr_active                              Y
pr_cgcode                             05
pr_code                          0171670
pr_dpcode                             19
pr_engname      JIRAAT YOUNG KALE 200 G.
pr_ggcode                            057
pr_market        JIRAPAT ยอดคะน้า 200 G.
pr_name          JIRAPAT ยอดคะน้า 200 G.
pr_puqty                               1
pr_sa_method                           1
pr_sucode1                         CM845
pr_suref3                              A
prtype                                 I
psqty                                  1
pstype                                 1
dtype: object

In [ ]:
df = Tester.loadFromCache()
df[df['cprcode']=='0217153'].iloc[0]

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


loading hashkey allData-hash
error downloading file An error occurred (404) when calling the HeadObject operation: Not Found
local loading error'list' object has no attribute 'get', loading remote hash
locding remtoe failed An error occurred (404) when calling the HeadObject operation: Not Found returning blank df


KeyError: 'cprcode'